In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv")
datax=df.sample(n=300, random_state=123)
datax.set_index('id',inplace=True)

datax=datax.iloc[:,-1941:]


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
def model_lstm(look_back):
    model=Sequential()
    model.add(LSTM(100, input_shape=(1, look_back), activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model
def model_loss(history):
    plt.figure(figsize=(8,4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Test Loss')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.legend(loc='upper right')
    plt.show();

def convert2matrix(data_arr, look_back):
   X, Y =[], []
   for i in range(len(data_arr)-look_back):
    d=i+look_back  
    X.append(data_arr[i:d,])
    Y.append(data_arr[d,])
   return np.array(X), np.array(Y)

import seaborn as sns

def prediction_plot(testY, test_predict):
      len_prediction=[x for x in range(len(testY))]
      plt.figure(figsize=(8,4))
      plt.plot(len_prediction, testY[:len(testY)], marker='.', label="Real")
      plt.plot(len_prediction, test_predict[:len(testY)], 'r', label="Pronostico")
      plt.tight_layout()
      sns.despine(top=True)
      plt.subplots_adjust(left=0.07)
      plt.ylabel('Pronostico Tendencia', size=15)
      plt.xlabel('Tamaño de la serie', size=15)
      plt.legend(fontsize=15)
      plt.show();
def smape(a, f):
    a = np.array([float(val) for val in a])
    f = np.array([float(val) for val in f])
    
    numerator = 2 * np.abs(f - a)
    denominator = np.abs(a) + np.abs(f)
    
    return 100 / len(a) * np.sum(numerator / denominator)

In [ ]:
smapeList=[]
predsList=[]
for i in range(300):
    
    #MA=datax.iloc[[i][0]].rolling(window=5).mean().dropna()
    nMA=datax.iloc[[i][0]].values
    data = nMA
    df_arr=data

    df_arr = np.reshape(df_arr, (-1, 1)) #LTSM requires more input features compared to RNN or DNN
    scaler = MinMaxScaler(feature_range=(0, 1))#LTSM is senstive to the scale of features
    df_arr = scaler.fit_transform(df_arr)

    train,test = train_test_split(df_arr,test_size=58,shuffle=False)
    look_back = 30

    trainX, trainY = convert2matrix(train, look_back)
    testX, testY = convert2matrix(test, look_back)
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    model=model_lstm(look_back)
    EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    history=model.fit(trainX,trainY, epochs=100, batch_size=30, verbose=1, validation_data=(testX,testY),callbacks=[EarlyStopping],shuffle=False)

    
    test_predict = model.predict(testX)
    # invert predictions
   
    test_predict = scaler.inverse_transform(test_predict)
    testY = scaler.inverse_transform(testY.reshape(-1,1))
    
    predsList.append(test_predict)
    smapeList.append(smape(testY, test_predict))
    print('-'*20,i,'-'*20)
    prediction_plot(testY, test_predict)
    


In [ ]:
preds=[]
for i in range(300):
    flat_preds = predsList[i].flatten()
    preds.append(flat_preds)

In [ ]:
# Crear un DataFrame
idsx=datax.index.tolist()
ids=idsx

df = pd.DataFrame(preds)
df['id'] = ids
df.set_index('id', inplace=True)
# Guardar el DataFrame en un archivo CSV
df.to_csv('preds_lstm_300sOrginal.csv', index=True)

In [21]:
p= sum(smapeList)/len(smapeList)
sp=round(p,2)
print('sMape promedio: ',sp)

sMape promedio:  138.7


In [ ]:
dfr= pd.DataFrame(smapeList)
dfr.to_csv("Results_lstm_300s.csv")